### Merge WWF's HydroBasins

* Purpose of script: This notebooks will copy the relevant files from raw ro process and merge the shapefiles of level 6
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20170802


## Preparation

make sure you are authorized to use AWS S3

## Origin of the WWF data

The Hydrosheds data has been downloaded from the [WWF Website](http://www.hydrosheds.org/download). A login is required for larger datasets. For Aqueduct we used the Standard version without lakes. Since the download is limited to 5GB we split up the download in two batches:  

1. Africa, North American Arctic, Central & South-east Asia, Australia & Oceania, Europe & Middle East
1. Greenland, North America & Caribbean, South America, Siberia

Download URLs (no longer valid)  
[link1](http://www.hydrosheds.org/tempdownloads/hydrosheds-3926b3742a77b18974ca.zip)  
[link2](http://www.hydrosheds.org/tempdownloads/hydrosheds-a69872e3f4059aea2434.zip)


The data was downloaded earlier but replicated here so the latest download data would be 2017/08/03 

The folders contain all levels but for this phase of Aqueduct we decided to use level 6. More information regarding this decision will be in the methodology document. 




## Script
copy the files from the raw data folder to the process data folder. The raw data folder contains pristine or untouched data and should not be used as a working directory



In [38]:
!cd /

sh: 0: getcwd() failed: No such file or directory


In [25]:
!aws s3 cp \
s3://wri-projects/Aqueduct30/rawData/WWF/HydroSheds30sComplete/HydrobasinsStandardAfr-Eu.zip \
s3://wri-projects/Aqueduct30/processData/02HydroBasinsV01/

copy: s3://wri-projects/Aqueduct30/rawData/WWF/HydroSheds30sComplete/HydrobasinsStandardAfr-Eu.zip to s3://wri-projects/Aqueduct30/processData/02HydroBasinsV01/HydrobasinsStandardAfr-Eu.zip


In [24]:
!aws s3 cp \
s3://wri-projects/Aqueduct30/rawData/WWF/HydroSheds30sComplete/HydrobasinsStandardGR-SI.zip \
s3://wri-projects/Aqueduct30/processData/02HydroBasinsV01/

copy: s3://wri-projects/Aqueduct30/rawData/WWF/HydroSheds30sComplete/HydrobasinsStandardGR-SI.zip to s3://wri-projects/Aqueduct30/processData/02HydroBasinsV01/HydrobasinsStandardGR-SI.zip


In [34]:
!mkdir /volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01

sh: 0: getcwd() failed: No such file or directory
mkdir: cannot create directory '/volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01': File exists


In [42]:
!aws s3 cp s3://wri-projects/Aqueduct30/processData/02HydroBasinsV01 /volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01 --recursive

download: s3://wri-projects/Aqueduct30/processData/02HydroBasinsV01/HydrobasinsStandardGR-SI.zip to ./HydrobasinsStandardGR-SI.zip
download: s3://wri-projects/Aqueduct30/processData/02HydroBasinsV01/HydrobasinsStandardAfr-Eu.zip to ./HydrobasinsStandardAfr-Eu.zip


Unzip shapefiles 

In [45]:
!cd /volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01

In [46]:
!pwd

/volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01


In [48]:
!find . -name '*.zip' -exec unzip {} \;

Archive:  ./HydrobasinsStandardAfr-Eu.zip
 extracting: hybas_af_lev00_v1c.zip  
 extracting: hybas_af_lev01-06_v1c.zip  
 extracting: hybas_af_lev01-12_v1c.zip  
 extracting: hybas_af_lev01_v1c.zip  
 extracting: hybas_af_lev02_v1c.zip  
 extracting: hybas_af_lev03_v1c.zip  
 extracting: hybas_af_lev04_v1c.zip  
 extracting: hybas_af_lev05_v1c.zip  
 extracting: hybas_af_lev06_v1c.zip  
 extracting: hybas_af_lev07_v1c.zip  
 extracting: hybas_af_lev08_v1c.zip  
 extracting: hybas_af_lev09_v1c.zip  
 extracting: hybas_af_lev10_v1c.zip  
 extracting: hybas_af_lev11_v1c.zip  
 extracting: hybas_af_lev12_v1c.zip  
 extracting: hybas_ar_lev00_v1c.zip  
 extracting: hybas_ar_lev01-06_v1c.zip  
 extracting: hybas_ar_lev01-12_v1c.zip  
 extracting: hybas_ar_lev01_v1c.zip  
 extracting: hybas_ar_lev02_v1c.zip  
 extracting: hybas_ar_lev03_v1c.zip  
 extracting: hybas_ar_lev04_v1c.zip  
 extracting: hybas_ar_lev05_v1c.zip  
 extracting: hybas_ar_lev06_v1c.zip  
 extracting: hybas_ar_lev07_v1c.zi

For now we will only use Level 6. Extracting in the same directory (/volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01)

In [53]:
!find / -name '*lev06_v1c.zip' -exec unzip -o {} \;

Archive:  /volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/hybas_au_lev06_v1c.zip
  inflating: hybas_au_lev06_v1c.dbf  
  inflating: hybas_au_lev06_v1c.prj  
  inflating: hybas_au_lev06_v1c.sbn  
  inflating: hybas_au_lev06_v1c.sbx  
  inflating: hybas_au_lev06_v1c.shp  
  inflating: hybas_au_lev06_v1c.shp.xml  
  inflating: hybas_au_lev06_v1c.shx  
  inflating: HydroBASINS_TechDoc_v1c.pdf  
Archive:  /volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/hybas_na_lev06_v1c.zip
  inflating: hybas_na_lev06_v1c.dbf  
  inflating: hybas_na_lev06_v1c.prj  
  inflating: hybas_na_lev06_v1c.sbn  
  inflating: hybas_na_lev06_v1c.sbx  
  inflating: hybas_na_lev06_v1c.shp  
  inflating: hybas_na_lev06_v1c.shp.xml  
  inflating: hybas_na_lev06_v1c.shx  
  inflating: HydroBASINS_TechDoc_v1c.pdf  
Archive:  /volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/hybas_gr_lev06_v1c.zip
  inflating: hybas_gr_lev06_v1c.dbf  
  inflating: hybas_gr_lev06_v1c.prj  
  inflating: hybas_gr_lev06_v1c.sbn  
  inflat

In [61]:
import os
import fiona

In [54]:
filePath = "/volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01"

In [56]:
files = os.listdir(filePath)

In [75]:
!pwd

/volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01


Create output folder

In [91]:
!mkdir /volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/output

In [92]:
meta = fiona.open('hybas_ar_lev06_v1c.shp').meta
with fiona.open('./output/hybas_lev06_v1c_merged_fiona_V01.shp', 'w', **meta) as output:
    for oneFile in files:    
        if oneFile.endswith(".shp"):
            print(oneFile)
            for features in fiona.open(oneFile):
                output.write(features)
    
    
    

hybas_sa_lev06_v1c.shp
hybas_as_lev06_v1c.shp
hybas_eu_lev06_v1c.shp
hybas_af_lev06_v1c.shp
hybas_au_lev06_v1c.shp
hybas_na_lev06_v1c.shp
hybas_ar_lev06_v1c.shp
hybas_gr_lev06_v1c.shp
hybas_si_lev06_v1c.shp


In [94]:
!tar -zcvf /volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/hybas_lev06_v1c_merged_fiona_V01.tar.gz /volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/output

tar: Removing leading `/' from member names
/volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/output/
/volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merged_fiona_V01.cpg
/volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merged_fiona_V01.dbf
/volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merged_fiona_V01.prj
/volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merged_fiona_V01.shp
/volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merged_fiona_V01.shx


In [97]:
!aws s3 cp /volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/output s3://wri-projects/Aqueduct30/temp/ --recursive

upload: output/hybas_lev06_v1c_merged_fiona_V01.prj to s3://wri-projects/Aqueduct30/temp/hybas_lev06_v1c_merged_fiona_V01.prj
upload: output/hybas_lev06_v1c_merged_fiona_V01.cpg to s3://wri-projects/Aqueduct30/temp/hybas_lev06_v1c_merged_fiona_V01.cpg
upload: output/hybas_lev06_v1c_merged_fiona_V01.dbf to s3://wri-projects/Aqueduct30/temp/hybas_lev06_v1c_merged_fiona_V01.dbf
upload: output/hybas_lev06_v1c_merged_fiona_V01.shx to s3://wri-projects/Aqueduct30/temp/hybas_lev06_v1c_merged_fiona_V01.shx
upload: output/hybas_lev06_v1c_merged_fiona_V01.shp to s3://wri-projects/Aqueduct30/temp/hybas_lev06_v1c_merged_fiona_V01.shp


The merged shapefile is stored in /volumes/data/Y2017M08D02_RH_Merge_HydroBasins_V01/output